<a href="https://colab.research.google.com/github/sashford/Honors-Thesis-Spencer-Ashford/blob/main/TorchSparseDenoiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!sudo apt-get install ffmpeg libavcodec-extra
!sudo apt-get install libopenblas-dev build-essential
!sudo apt-get install python3 python-dev python3-dev
!apt install subversion
!svn checkout https://github.com/sashford/Honors-Thesis-Spencer-Ashford.git
!pip3 install Cmake
!pip3 install ninja
!pip install git+https://github.com/NVIDIA/MinkowskiEngine.git -v

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
libavcodec-extra is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
libopenblas-dev is already the newest version (0.3.8+ds-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
python3 is already the newest version (3.8.2-0ubuntu2).
python3-dev is already the newest version (3.8.2-0ubuntu2).
python-dev-is-python2 is already the newest version (2.7.17-4).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading pa

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from torch import nn
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
import torch.optim as optim
from sklearn import linear_model

import MinkowskiEngine as ME

import librosa
from IPython.display import display, Audio

# Plotter helper

In [ ]:
azi = 120
binsA = 90
minR = 1
maxR = 30
binsR = 512

def plotter(raw_input, target_input, processed_input=None):

  if processed_input != None:
    num_inputs = 3
    fig, (ax1, ax3, ax2) = plt.subplots(1,num_inputs, subplot_kw=dict(projection='polar'))
  else:
    num_inputs = 2
    fig, (ax1, ax3) = plt.subplots(1,num_inputs, subplot_kw=dict(projection='polar'))

  ax1.set_title("Noisy")
  ax1.set_theta_zero_location("N")
  ax1.set_thetamin(-azi/2)
  ax1.set_thetamax(azi/2)
  ax1.grid(False)

  if num_inputs == 3:
    ax2.set_title("Denoised")
    ax2.set_theta_zero_location("N")
    ax2.set_thetamin(-azi/2)
    ax2.set_thetamax(azi/2)
    ax2.grid(False)

  ax3.set_title("Noiseless")
  ax3.set_theta_zero_location("N")
  ax3.set_thetamin(-azi/2)
  ax3.set_thetamax(azi/2)
  ax3.grid(False)

  theta = np.linspace(-azi/2, azi/2, binsA) * np.pi / 180
  r = np.linspace(minR, maxR, binsR)
  T, R = np.meshgrid(theta,r)
  z = np.zeros_like(T)

  plot_noise = ax1.pcolormesh(T, R, z, cmap='gray', shading='auto', vmin=0, vmax=1)
  if num_inputs == 3:
    plot_denoised = ax2.pcolormesh(T, R, z, cmap='gray', shading='auto', vmin=0, vmax=1)
  plot_noiseless = ax3.pcolormesh(T, R, z, cmap='gray', shading='auto', vmin=0, vmax=1)
  plt.tight_layout()

  if isinstance(raw_input, tuple):
    raw_input = raw_input[0]
  if isinstance(target_input, tuple):
    target_input = target_input[0]
  if num_inputs == 3 and isinstance(processed_input, tuple):
    processed_input = processed_input[0]

  plot_noise.set_array(raw_input.numpy().ravel())
  if num_inputs == 3:
    plot_denoised.set_array(processed_input.numpy().ravel())
  plot_noiseless.set_array(target_input.numpy().ravel())

  plt.show()

# Class Definitions

In [ ]:
from collections import deque
def npy_loader(path):
    sonar_data = torch.from_numpy(np.load(path))
    return sonar_data

def generate_sparse_tensor(input):
  input_sparse = input.to_sparse(layout=torch.sparse_coo)
  input_values = input_sparse.values()
  input_indices = input_sparse.indices()

torch.set_printoptions(precision=5)

class Indices(deque):
  def __init__(self):
    super().__init__(self)
    for i in range(batch_size):
      self.append(None)

  def next_index(self, index):
    self.append(index)
    self.popleft()


class SonarDataset(Dataset):
  def __init__(self):

    root_folder = "/content/Honors-Thesis-Spencer-Ashford.git/trunk/dataset/"
    self.noise_folder = datasets.DatasetFolder(root=root_folder + 'noise', loader=npy_loader, extensions=['.npy'])
    self.noiseless_folder = datasets.DatasetFolder(root=root_folder + 'noiseless',loader=npy_loader, extensions=['.npy'])
    self.index = None
    self.batch_indices = Indices()

  def get_noiseless(self, index):
    return self.noiseless_folder[index][0]

  def get_noisy(self, index):
    return self.noise_folder[index][0]

  def batch_targets(self):
    batch_target = []
    for index in self.batch_indices:
      target = self.get_noiseless(index)

      batch_target.append(target.unsqueeze(0))
    return torch.cat(batch_target,0).unsqueeze(1)

  def __getitem__(self,index):
    self.batch_indices.next_index(index)
    noisy = self.get_noisy(index)

    if noisy.is_cuda:
      return noisy
    else:
      return noisy.cuda()

  def __len__(self):
    return len(self.noise_folder)

In [ ]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder_conv = nn.Sequential(
        nn.Flatten(0),
        nn.Linear(in_features= binsA * binsR * batch_size, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=128),
        nn.ReLU(),
    )

    self.decoder_conv = nn.Sequential(
        nn.Linear(in_features=128, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features= binsA * binsR * batch_size),
        nn.ReLU(),
        nn.Unflatten(dim=0, unflattened_size=[batch_size, binsR, binsA])
    )

  def forward(self,x):
    encoded = self.encoder_conv(x)
    return self.decoder_conv(encoded)

In [ ]:
def sparse_loss(autoencoder, images):
  loss = 0
  values = images
  for i in range(len(model_children)):
    values = F.relu((model_children[i](values)))
    loss += torch.mean(torch.abs(values))
  return loss



# Initialization and Parameters

In [ ]:
batch_size = 4
lr = 0.001
reg_param = 0.001
sonar_data = SonarDataset()

train_data, val_data = random_split(sonar_data, [0.85, 0.15])

train_loader = DataLoader(train_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

model = AutoEncoder()
model.cuda()

model_children = list(model.children())

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()

# Training/Validation

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer):
  model.train()
  train_loss = []

  loader = iter(dataloader)
  for i, input in enumerate(loader):
    if input.size()[0] == batch_size:
      optimizer.zero_grad()
      print(input.size())
      decoded_data = model(input)
      noiseless_image = sonar_data.batch_targets().cuda()

      loss = loss_fn(decoded_data, noiseless_image) + reg_param * sparse_loss(model, noiseless_image)
      loss.backward()
      optimizer.step()

      print(f'\t partial train loss {i} (single batch): {loss.data}')#| sizes: {decoded_data.size()}. {np.shape(noiseless_image)}')
      train_loss.append(loss.detach().cpu().numpy())
      input.cpu()

    torch.cuda.empty_cache()

  return np.mean(train_loss)

In [ ]:
def test_epoch(model, dataloader, loss_fn):
  model.eval()

  with torch.no_grad():
    conc_out = []
    conc_label = []
    loader = iter(dataloader)
    for i, input in enumerate(loader):
      if input.size()[0] == batch_size:
        print(input.size())
        decoded_data = model(input)
        noiseless_images = sonar_data.batch_targets().cuda()

        conc_out.append(decoded_data)
        conc_label.append(noiseless_images)

        output = decoded_data
        label = noiseless_images
        # print(f"out: {output.size()} label: {label.size()}")
        input.cpu()

    conc_out = torch.cat(conc_out)
    conc_label = torch.cat(conc_label)
    # print(conc_out.size())
    # print(conc_label.size())

    val_loss = loss_fn(conc_out, conc_label)
    torch.cuda.empty_cache()

  return val_loss.data

In [ ]:
def combine_tensor(tensor):
  comb_tensor = []

  for i in range(batch_size):
    comb_tensor.append(tensor)

  comb_tensor = torch.cat(comb_tensor, 0)
  return comb_tensor

def convert_to_dense(stensor):


def plot_outputs(model):
  index = np.random.randint(0,len(val_data))
  input = val_data[index]

  model.eval()

  with torch.no_grad():
    decoded_data = model(input)

    denoised = decoded_data[0]

    plotter(sonar_data.get_noisy(index), sonar_data.get_noiseless(index), denoised.cpu())

# Main Loop

In [ ]:
num_epochs = 50
history_da={'train_loss':[], 'val_loss':[]}

for epoch in range(num_epochs):
  print(f'EPOCH {epoch + 1}/{num_epochs}')
  train_loss = train_epoch(
      model=model,
      dataloader=train_loader,
      loss_fn=loss_fn,
      optimizer=optimizer
  )
  print("beginning validation")
  val_loss = test_epoch(
      model=model,
      dataloader=val_loader,
      loss_fn=loss_fn
  )
  print("begin plotting")
  history_da['train_loss'].append(train_loss)
  history_da['val_loss'].append(val_loss)
  print('\n EPOCH {}/{} \t train loss {:.3f} \t val loss {:.3f}'.format(epoch + 1, num_epochs,train_loss,val_loss))

  plot_outputs(model)

EPOCH 1/50
torch.Size([4, 512, 90])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 1, 512, 90])) that is different to the input size (torch.Size([4, 512, 90])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


	 partial train loss 0 (single batch): 0.0007565435371361673
torch.Size([4, 512, 90])
	 partial train loss 1 (single batch): 0.0005774709861725569
torch.Size([4, 512, 90])
	 partial train loss 2 (single batch): 0.0005520402337424457
torch.Size([4, 512, 90])
	 partial train loss 3 (single batch): 0.0004997157375328243
torch.Size([4, 512, 90])
	 partial train loss 4 (single batch): 0.0002901458356063813
torch.Size([4, 512, 90])
	 partial train loss 5 (single batch): 0.0006618062616325915
torch.Size([4, 512, 90])
	 partial train loss 6 (single batch): 0.0003911512903869152
torch.Size([4, 512, 90])
	 partial train loss 7 (single batch): 0.00031051994301378727
torch.Size([4, 512, 90])
	 partial train loss 8 (single batch): 0.0005069591570645571
torch.Size([4, 512, 90])
	 partial train loss 9 (single batch): 0.0003611561260186136
torch.Size([4, 512, 90])
	 partial train loss 10 (single batch): 0.00028238308732397854
torch.Size([4, 512, 90])
	 partial train loss 11 (single batch): 0.000276763

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([176, 1, 512, 90])) that is different to the input size (torch.Size([176, 512, 90])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


begin plotting

 EPOCH 1/50 	 train loss 0.000 	 val loss 0.000


RuntimeError: ignored

In [ ]:
# for playing wav file
sound, rate = librosa.load("/content/Honors-Thesis-Spencer-Ashford.git/trunk/misc/EndTrainingLoop.wav")
print('playing sound using  pydub')
display(Audio(sound, rate=rate, autoplay=True))